In [1]:
import os
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import pandas as pd
import pickle
import math
from pydub import AudioSegment

In [2]:
"""getting directories for wav"""
os.chdir("./VCTK-Corpus") #path for the dataset goes here
wav_dir = (os.getcwd()+'/wav48')

In [3]:
"""getting all the directories containing wav files"""
directories = []
for dirpaths,dirnames,filenames in os.walk(wav_dir):
    directories.append(dirpaths)
directories = directories[1:] #ignoring current directory

In [4]:
"""creating list of paths for all wav files"""
files_wav = []
for i in directories:
    for dirpaths,dirnames,filenames in os.walk(i):
        for j in filenames:
            files_wav.append(i+'/'+j.split(".")[0]+".wav")

In [5]:
%%time
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

for i in files_wav:
    try:
        sound = AudioSegment.from_file(i, format="wav")
        start_trim = detect_leading_silence(sound,-35)
        end_trim = detect_leading_silence(sound.reverse(),-45)
        duration = len(sound)
        if duration <800:
            print i,duration
        trimmed_sound = sound[start_trim:duration-end_trim]
        trimmed_sound.export(i,format = "wav")
        
    except:
        print i

CPU times: user 1min 10s, sys: 10.8 s, total: 1min 21s
Wall time: 3min 40s
